In [1]:
import os
os.chdir("../")

In [2]:
os.listdir()

['setup.py',
 'setup.cfg',
 'requirements_dev.txt',
 'logs',
 'env',
 'tests',
 'stg1.py',
 'requiremets.txt',
 'Template.py',
 '.gitignore',
 'pyproject.toml',
 'research',
 'artifacts',
 'artifact',
 'configs',
 '.venv',
 'dvc.yaml',
 'params.yaml',
 'README.md',
 'src',
 'init_setup.sh',
 '.git',
 'tox.ini',
 'LICENSE',
 'requirements.txt',
 'art',
 '.github']

In [3]:
from dataclasses import dataclass
from pathlib import Path
@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    kernel_layers1: int
    kernel_layers2: int
    kernel_layers3: int
    params_classes: int
    params_final_output_layers: int
    params_augmentation: bool
    params_image_size: list
    params_batch_size: int
    params_include_top: bool
    params_epochs: int

In [4]:
from ASL_alphabet.constants import *
from ASL_alphabet.utils import read_yaml, create_directories

In [5]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
            config = self.config.prepare_base_model
            
            create_directories([config.root_dir])

            prepare_base_model_config = PrepareBaseModelConfig(
                root_dir=Path(config.root_dir),
                base_model_path=Path(config.base_model_path),
                updated_base_model_path=Path(config.updated_base_model_path),
                kernel_layers1= self.params.KERNEL_LAYERS1,
                kernel_layers2= self.params.KERNEL_LAYERS2,
                kernel_layers3= self.params.KERNEL_LAYERS3,
                params_classes= self.params.CLASSES,
                params_final_output_layers= self.params.FINAL_OUTPUT_LAYERS,
                params_augmentation= self.params.AUGMENTATION,
                params_image_size= self.params.IMAGE_SIZE,
                params_batch_size= self.params.BATCH_SIZE,
                params_include_top= self.params.INCLUDE_TOP,
                params_epochs= self.params.EPOCHS,
         
            )

            return prepare_base_model_config

In [6]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

2023-07-10 20:47:33.912917: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-10 20:47:34.429200: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [9]:
import os
import urllib.request as request
from ASL_alphabet.entity import PrepareBaseModelConfig
import tensorflow as tf
from pathlib import Path


class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config

    def create_base_model(self):

        self.model = tf.keras.Sequential(
            [
                tf.keras.Input(shape=self.config.params_image_size),
                tf.keras.layers.Conv2D(
                        self.config.kernel_layers1,(3, 3), 
                        strides=1,padding='same', 
                        activation="relu"
                    ),
                tf.keras.layers.BatchNormalization(),
                tf.keras.layers.MaxPooling2D(
                    pool_size=(2, 2),
                    strides=2,
                    padding='same'
                ),
                tf.keras.layers.Conv2D(self.config.kernel_layers2,(3,3),strides=1, padding='same', activation="relu"),
                tf.keras.layers.Dropout(
                    0.2
                ),
                tf.keras.layers.BatchNormalization(),
                tf.keras.layers.MaxPooling2D(
                    pool_size=(2, 2),
                    strides=2,
                    padding='same'
                ),
                tf.keras.layers.Conv2D(self.config.kernel_layers3,(3,3),strides=1, padding='same', activation="relu"),
                tf.keras.layers.BatchNormalization(),
                tf.keras.layers.MaxPooling2D(
                    pool_size=(2, 2),
                    strides=2,
                    padding='same'
                ),
            ]
        )
 
        self.save_model(path=self.config.base_model_path, model=self.model)
        
       



    @staticmethod
    def _prepare_full_model(model, classes,final_output_layers):
   
        flatten_in = tf.keras.layers.Flatten()(model.output)
        flatten_in = tf.keras.layers.Dense(units=classes,activation='relu')(flatten_in)
        flatten_in = tf.keras.layers.Dropout(0.3)(flatten_in)
        prediction = tf.keras.layers.Dense(
            units=final_output_layers,
            activation="softmax"
        )(flatten_in)

        full_model = tf.keras.models.Model(
            inputs=model.input,
            outputs=prediction
        )

        full_model.compile(
            optimizer=tf.keras.optimizers.Adam(),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )

        full_model.summary()
        return full_model

    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            final_output_layers=self.config.params_final_output_layers,

        )

        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

In [10]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.create_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e

[2023-07-10 21:08:03,841: INFO: utility]: yaml file: configs/config.yaml loaded successfully
[2023-07-10 21:08:03,843: INFO: utility]: yaml file: params.yaml loaded successfully
[2023-07-10 21:08:03,843: INFO: utility]: created directory at: artifacts
[2023-07-10 21:08:03,844: INFO: utility]: created directory at: artifacts/prepare_base_model
[2023-07-10 21:08:03,917: WARNING: saving_utils]: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d_3 (Conv2D)           (None, 28, 28, 75)        750       
                                                                 
 batch_normalization_3 (Bat  (None, 28, 28,

/home/lang_chain/Documents/ASL_alphabet/.venv/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
ls

/bin/bash: /home/lang_chain/anaconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
kaggle.json  logs/  stg2.ipynb
